# Analyze Fine-Tuned EfficientPhys Models

## Load the Predictions

In [ ]:
import os
import numpy as np
import pandas as pd
import respiration.utils as utils

evaluation_dir = os.path.join('..', 'evaluation', 'efficeint_phys_fine_tuned')
evaluation_path = os.path.join(evaluation_dir, 'predictions.csv')

prediction = pd.read_csv(evaluation_path)
prediction['signal'] = prediction['signal'].apply(eval).apply(np.array)

In [ ]:
prediction.head()

In [ ]:
prediction['id'].unique()

## Load the model specifications

In [ ]:
manifest_path = os.path.join(evaluation_dir, 'manifests.json')
manifests = utils.read_json(manifest_path)
manifests = {manifest['id']: manifest for manifest in manifests}

## Get the Ground Truth RR Signal

In [ ]:
ground_truth_file = os.path.join(os.getcwd(), '..', 'evaluation', 'ground_truth.csv')
ground_truth = pd.read_csv(ground_truth_file)
ground_truth['signal'] = ground_truth['signal'].apply(eval).apply(np.array)
ground_truth.head()

## Analyze the Performance of the Model

In [ ]:
import scipy.signal as signal
import respiration.preprocessing as preprocessing


def get_ground_truth(subject, setting, sample_rate: int, length: int) -> np.ndarray:
    subject_gt = ground_truth[(ground_truth['subject'] == subject) &
                              (ground_truth['setting'] == setting)].iloc[0]

    gt_signal = subject_gt['signal']

    # Normalize the signal
    gt_signal = preprocessing.normalize_signal(gt_signal)

    # Resample the signal to sample_rate
    gt_signal = signal.resample(gt_signal, int(len(gt_signal) * sample_rate / subject_gt['sample_rate']))

    # Cut the signal to the same length as the prediction
    gt_signal = gt_signal[:length]

    return gt_signal

In [ ]:
def get_prediction(subject, setting, model_id) -> tuple[np.ndarray, int]:
    subject_pred = prediction[(prediction['id'] == model_id) &
                              (prediction['subject'] == subject) &
                              (prediction['setting'] == setting)].iloc[0]

    sample_rate = subject_pred['sample_rate']

    pred_signal = subject_pred['signal']
    # pred_signal = preprocessing.standard_processing(pred_signal, sample_rate)

    return pred_signal, sample_rate

In [ ]:
import matplotlib.pyplot as plt
import respiration.dataset as repository

dataset = repository.from_default()

subject = 'Proband25'
setting = '101_natural_lighting'
model_id = '20240504_163423'

pred_signal, sample_rate = get_prediction(subject, setting, model_id)
gt_signal = get_ground_truth(subject, setting, sample_rate, len(pred_signal))

In [ ]:
manifests[model_id]['loss_fn']

In [ ]:
# Plot the signals
fig, ax = plt.subplots(2, 1, figsize=(20, 10))

ax[0].plot(pred_signal, label='Prediction')
ax[0].set_title('Prediction')
ax[0].legend()

ax[1].plot(gt_signal, label='Ground Truth')
ax[1].set_title('Ground Truth')
ax[1].legend()

In [ ]:
import respiration.utils as utils
import respiration.analysis as analysis

compare = analysis.SignalCompare(
    pred_signal,
    sample_rate,
    gt_signal,
    sample_rate,
    # normalize_signal=False,
    # filter_signal=False,
    # detrend_tarvainen=False,
)

utils.pretty_print(compare.distances())

In [ ]:
compare_results = []

for model_id, manifest in manifests.items():
    print(model_id)

    for (subject, setting) in manifest['testing_scenarios']:
        pred_signal, sample_rate = get_prediction(subject, setting, model_id)
        gt_signal = get_ground_truth(subject, setting, sample_rate, len(pred_signal))

        compare = analysis.SignalCompare(
            pred_signal,
            sample_rate,
            gt_signal,
            sample_rate,
        )

        entry = compare.distances()
        entry['model_id'] = model_id
        entry['subject'] = subject
        entry['setting'] = setting
        compare_results.append(entry)

compare_results = pd.DataFrame(compare_results)

In [ ]:
compare_results

In [ ]:
model_scores = []

for model_id, manifest in manifests.items():
    model_results = compare_results[compare_results['model_id'] == model_id]

    model_scores.append({
        'model_id': model_id,
        'loss_fn': manifest['loss_fn'],
        'optimizer': manifest['optimizer'],
        'learning_rate': manifest['learning_rate'],
        'chunk_size': manifest['chunk_size'],
        'epochs': manifest['epochs'],
        'mean_nfcp_error': model_results['nfcp_error'].mean(),
        'mean_distance_dtw': model_results['distance_dtw'].mean(),
        'mean_distance_mse': model_results['distance_mse'].mean(),
        'mean_distance_pearson': model_results['distance_pearson'].mean(),
    })

model_scores = pd.DataFrame(model_scores)

In [ ]:
model_scores